In [4]:
import sys
import os
import numpy as np
import pandas as pd
import pickle
import math
from astropy.io import fits
from sedpy.observate import load_filters
from prospect import prospect_args
from prospect.fitting import fit_model
from prospect.models.sedmodel import SpecModel, LineSpecModel
from prospect.io import write_results as writer
from astropy.cosmology import Planck15 as cosmo
from astropy.table import Table
from prospect.likelihood import NoiseModel
from prospect.likelihood.kernels import Uncorrelated
from prospect.sources.galaxy_basis import CSPSpecBasis

In [22]:
path_wdir = "/Users/amanda/Desktop/Paper/technical/"
path_data = os.path.join(path_wdir, "data/")
path_output = os.path.join(path_wdir, "prospector/")




In [16]:
file = [0,1,2,3,4,8]

In [17]:
filename = 'test2.pickle'

# open the file in write mode and save the dictionary as a pickle
with open(path_output + filename, 'wb') as f:
    pickle.dump(file, f)


In [18]:

# open the file in read mode and load the pickled dictionary
with open(path_output + filename, 'rb') as f:
    output = pickle.load(f)

In [19]:
output

[0, 1, 2, 3, 4, 8]

In [28]:
from astropy.io import fits
import pandas as pd
import numpy as np

# Read the CSV file using pandas
# It is important that the data itself does not contain any data type that is not convertable to float.
# If it does, a dictionary is required to translate the data.
df = pd.read_csv(path_data+'galaxies_nostring.csv')
df = df.astype(float)

# Convert the dataframe to a numpy structured array
data = df.to_records(index=False)

# Create a new FITS binary table HDU
hdu = fits.BinTableHDU.from_columns(data)

# Write the FITS file
hdu.writeto(path_data+'galaxies_nostring.fits', overwrite=True)

In [112]:

num_to_id_translate = {'3307':'J003601+003307',
                '15440':'J004743+015440',
                '223':'J011309+000223',
                '52044':'J012217+052044',
                '145935':'J012910+145935',
                '414608':'J072326+414608',
                '472607':'J080425+472607',
                '414146':'J081112+414146',
                '211459':'J081409+211459',
                '182052':'J082652+182052',
                '480541':'J083440+480541',
                '392925':'J090918+392925',
                '183108':'J091113+183108',
                '523960':'J091207+523960',
                '505009':'J091208+505009',
                '315221':'J091703+315221',
                '395714':'J092552+395714',
                '510925':'J093355+510925',
                '593244':'J094001+593244',
                '405249':'J095236+405249',
                '235709':'J095700+235709',
                '202508':'J095838+202508',
                '523251':'J101401+523251',
                '633308':'J102615+633308',
                '635317':'J103344+635317',
                '452718':'J103816+452718',
                '474357':'J105117+474357',
                '523753':'J105331+523753',
                '485724':'J105525+485724',
                '475204':'J110452+475204',
                '412052':'J112224+412052',
                '524509':'J112848+524509',
                '493525':'J112933+493525',
                '651341':'J113304+651341',
                '312559':'J115855+312559',
                '382422':'J115959+382422',
                '305326':'J120934+305326',
                '453930':'J121915+453930',
                '63556':'J123519+063556',
                '212716':'J124033+212716',
                '21540':'J124423+021540',
                '444902':'J124619+444902',
                '123403':'J124835+123403',
                '464535':'J124911+464535',
                '255609':'J125503+255609',
                '410221':'J125718+410221',
                '451057':'J130059+451057',
                '510451':'J130128+510451',
                '422638':'J130559+422638',
                '214817':'J131037+214817',
                '104739':'J131419+104739',
                '510309':'J131904+510309',
                '421824':'J132633+421824',
                '573315':'J132937+573315',
                '112848':'J134559+112848',
                '662438':'J135008+662438',
                '612115':'J140333+612115',
                '434435':'J141013+434435',
                '461937':'J144010+461937',
                '370512':'J151707+370512',
                '572442':'J154050+572442',
                '403325':'J155945+403325',
                '81959':'J160437+081959',
                '313054':'J164607+313054',
                '495751':'J164849+495751',
                '542133':'J172010+542133'}
id_to_num_translate = {v: k for k, v in num_to_id_translate.items()}


In [113]:
data = fits.open(os.path.join(path_data, 'galaxies_nostring.fits'))
el_cat = data[1].data

In [114]:
el_cat['OBJID']

array([3.30700e+03, 1.54400e+04, 2.23000e+02, 5.20440e+04, 1.45935e+05,
       4.14608e+05, 4.72607e+05, 4.14146e+05, 2.11459e+05, 1.82052e+05,
       4.80541e+05, 3.92925e+05, 1.83108e+05, 5.23960e+05, 5.05009e+05,
       3.15221e+05, 3.95714e+05, 5.10925e+05, 5.93244e+05, 4.05249e+05,
       2.35709e+05, 2.02508e+05, 5.23251e+05, 6.33308e+05, 6.35317e+05,
       4.52718e+05, 4.74357e+05, 5.23753e+05, 4.85724e+05, 4.75204e+05,
       4.12052e+05, 5.24509e+05, 4.93525e+05, 6.51341e+05, 3.12559e+05,
       3.82422e+05, 3.05326e+05, 4.53930e+05, 6.35560e+04, 2.12716e+05,
       2.15400e+04, 4.44902e+05, 1.23403e+05, 4.64535e+05, 2.55609e+05,
       4.10221e+05, 4.51057e+05, 5.10451e+05, 4.22638e+05, 2.14817e+05,
       1.04739e+05, 5.10309e+05, 4.21824e+05, 5.73315e+05, 1.12848e+05,
       6.62438e+05, 6.12115e+05, 4.34435e+05, 4.61937e+05, 3.70512e+05,
       5.72442e+05, 4.03325e+05, 8.19590e+04, 3.13054e+05, 4.95751e+05,
       5.42133e+05])

In [115]:
str(int(el_cat['OBJID'][0]))

'3307'

In [116]:
ID = str(int(el_cat['OBJID'][0]))
num_to_id_translate[ID]

'J003601+003307'

In [121]:
for i in range(len((el_cat['OBJID']))):
    ID = num_to_id_translate[str(int(el_cat['OBJID'][i]))]
    print(ID)

J003601+003307
J004743+015440
J011309+000223
J012217+052044
J012910+145935
J072326+414608
J080425+472607
J081112+414146
J081409+211459
J082652+182052
J083440+480541
J090918+392925
J091113+183108
J091207+523960
J091208+505009
J091703+315221
J092552+395714
J093355+510925
J094001+593244
J095236+405249
J095700+235709
J095838+202508
J101401+523251
J102615+633308
J103344+635317
J103816+452718
J105117+474357
J105331+523753
J105525+485724
J110452+475204
J112224+412052
J112848+524509
J112933+493525
J113304+651341
J115855+312559
J115959+382422
J120934+305326
J121915+453930
J123519+063556
J124033+212716
J124423+021540
J124619+444902
J124835+123403
J124911+464535
J125503+255609
J125718+410221
J130059+451057
J130128+510451
J130559+422638
J131037+214817
J131419+104739
J131904+510309
J132633+421824
J132937+573315
J134559+112848
J135008+662438
J140333+612115
J141013+434435
J144010+461937
J151707+370512
J154050+572442
J155945+403325
J160437+081959
J164607+313054
J164849+495751
J172010+542133


In [122]:
list=[]
for i in range(len((el_cat['OBJID']))):
    ID = num_to_id_translate[str(int(el_cat['OBJID'][i]))]
    list.append(ID)

In [119]:
str(el_cat['OBJID'])

'[3.30700e+03 1.54400e+04 2.23000e+02 5.20440e+04 1.45935e+05 4.14608e+05\n 4.72607e+05 4.14146e+05 2.11459e+05 1.82052e+05 4.80541e+05 3.92925e+05\n 1.83108e+05 5.23960e+05 5.05009e+05 3.15221e+05 3.95714e+05 5.10925e+05\n 5.93244e+05 4.05249e+05 2.35709e+05 2.02508e+05 5.23251e+05 6.33308e+05\n 6.35317e+05 4.52718e+05 4.74357e+05 5.23753e+05 4.85724e+05 4.75204e+05\n 4.12052e+05 5.24509e+05 4.93525e+05 6.51341e+05 3.12559e+05 3.82422e+05\n 3.05326e+05 4.53930e+05 6.35560e+04 2.12716e+05 2.15400e+04 4.44902e+05\n 1.23403e+05 4.64535e+05 2.55609e+05 4.10221e+05 4.51057e+05 5.10451e+05\n 4.22638e+05 2.14817e+05 1.04739e+05 5.10309e+05 4.21824e+05 5.73315e+05\n 1.12848e+05 6.62438e+05 6.12115e+05 4.34435e+05 4.61937e+05 3.70512e+05\n 5.72442e+05 4.03325e+05 8.19590e+04 3.13054e+05 4.95751e+05 5.42133e+05]'

In [123]:
len(list)

66